Khai báo thư viện

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
# Cấu hình
IMG_SIZE = 128  # Tăng lên 128 cho transfer learning
BATCH_SIZE = 16
EPOCHS = 100

Chuẩn bị dữ liệu

In [ ]:
def create_dataset(data_dir):
    """Tạo dataset từ thư mục images/train và images/valid"""
    train_dir = Path(data_dir) / 'train'
    valid_dir = Path(data_dir) / 'valid'
    
    if not train_dir.exists():
        raise ValueError(f"Thư mục train không tồn tại: {train_dir}")
    if not valid_dir.exists():
        raise ValueError(f"Thư mục valid không tồn tại: {valid_dir}")
    
    def load_from_dir(directory):
        image_paths = []
        labels = []
        
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
            image_paths.extend(directory.glob(ext))
        
        for img_path in image_paths:
            filename = img_path.stem.lower()
            if 'fall' in filename and 'not' not in filename:
                labels.append(1)  # Fall
            else:
                labels.append(0)  # Not fall
        
        return [str(p) for p in image_paths], np.array(labels)
    
    train_paths, train_labels = load_from_dir(train_dir)
    val_paths, val_labels = load_from_dir(valid_dir)
    
    print(f"\n{'='*60}")
    print(f"TRAINING SET: {len(train_paths)} images")
    print(f"  - Fall: {sum(train_labels)} ({sum(train_labels)/len(train_labels)*100:.1f}%)")
    print(f"  - Not Fall: {len(train_labels)-sum(train_labels)} ({(len(train_labels)-sum(train_labels))/len(train_labels)*100:.1f}%)")
    print(f"\nVALIDATION SET: {len(val_paths)} images")
    print(f"  - Fall: {sum(val_labels)} ({sum(val_labels)/len(val_labels)*100:.1f}%)")
    print(f"  - Not Fall: {len(val_labels)-sum(val_labels)} ({(len(val_labels)-sum(val_labels))/len(val_labels)*100:.1f}%)")
    print(f"{'='*60}\n")
    
    if len(train_paths) == 0:
        raise ValueError(f"Không tìm thấy ảnh trong {train_dir}")
    if len(val_paths) == 0:
        raise ValueError(f"Không tìm thấy ảnh trong {valid_dir}")
    
    return train_paths, train_labels, val_paths, val_labels

Tiền xử lý hình ảnh

In [ ]:
def load_and_preprocess_image(path, label):
    """Load và tiền xử lý ảnh"""
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0
    return img, label

def augment_image(img, label):
    """Data augmentation"""
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, 0.2)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    img = tf.clip_by_value(img, 0.0, 1.0)
    return img, label

def create_tf_dataset(paths, labels, is_training=True):
    """Tạo TensorFlow dataset"""
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_training:
        dataset = dataset.shuffle(1000)
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.repeat()
    
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

Khai báo các kiến trúc mô hình

In [ ]:
def create_simple_cnn():
    """Mô hình CNN đơn giản - nhỏ nhất"""
    model = keras.Sequential([
        layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
        
        layers.Conv2D(16, 3, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(2),
        
        layers.Conv2D(32, 3, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(2),
        
        layers.Conv2D(64, 3, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.GlobalAveragePooling2D(),
        
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ], name='SimpleCNN')
    
    return model

In [ ]:
def create_mobilenetv2_model():
    """MobileNetV2 - Transfer Learning (RECOMMENDED cho ESP32)"""
    base_model = MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet',
        alpha=0.35  # Version nhỏ nhất của MobileNetV2
    )
    
    # Freeze base model
    base_model.trainable = False
    # Thêm các lớp tùy chỉnh phía trên base model
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ], name='MobileNetV2')
    
    return model, base_model

In [ ]:
def create_efficientnet_model():
    """EfficientNetB0 - Hiệu quả cao"""
    base_model = EfficientNetB0(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    
    base_model.trainable = False
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ], name='EfficientNetB0')
    
    return model, base_model

In [ ]:
def create_custom_cnn():
    """Custom CNN - Cân bằng giữa kích thước và hiệu suất"""
    model = keras.Sequential([
        layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
        
        # Block 1
        layers.Conv2D(24, 3, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        
        # Block 2
        layers.Conv2D(48, 3, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        
        # Block 3
        layers.Conv2D(96, 3, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),
        
        # Block 4
        layers.Conv2D(128, 3, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.GlobalAveragePooling2D(),
        
        # Classifier
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ], name='CustomCNN')
    
    return model

Huấn luyện các mô hình

In [ ]:
def plot_training_history(history, model_name):
    """Vẽ biểu đồ training history"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss
    ax1.plot(history.history['loss'], label='Train Loss', linewidth=2)
    ax1.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title(f'{model_name} - Loss', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Accuracy
    ax2.plot(history.history['accuracy'], label='Train Acc', linewidth=2)
    ax2.plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy', fontsize=12)
    ax2.set_title(f'{model_name} - Accuracy', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    filename = f'history_{model_name.lower().replace(" ", "_")}.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    print(f"  📊 Saved: {filename}")
    plt.close()

def train_model(model_name, data_dir):
    """Train mô hình"""
    print(f"\n{'='*60}")
    print(f"🚀 TRAINING: {model_name}")
    print(f"{'='*60}\n")
    
    # Load data
    train_paths, train_labels, val_paths, val_labels = create_dataset(data_dir)
    
    steps_per_epoch = len(train_paths) // BATCH_SIZE
    validation_steps = len(val_paths) // BATCH_SIZE
    
    train_ds = create_tf_dataset(train_paths, train_labels, is_training=True)
    val_ds = create_tf_dataset(val_paths, val_labels, is_training=False)
    
    # Create model
    base_model = None
    if model_name == 'MobileNetV2':
        model, base_model = create_mobilenetv2_model()
    elif model_name == 'EfficientNetB0':
        model, base_model = create_efficientnet_model()
    elif model_name == 'CustomCNN':
        model = create_custom_cnn()
    else:  # SimpleCNN
        model = create_simple_cnn()
    
    model.summary()
    
    # Class weights
    total = len(train_labels)
    n_fall = sum(train_labels)
    n_not_fall = total - n_fall
    
    if n_fall > 0 and n_not_fall > 0:
        weight_for_fall = (1 / n_fall) * (total / 2.0)
        weight_for_not_fall = (1 / n_not_fall) * (total / 2.0)
        class_weight = {0: weight_for_not_fall, 1: weight_for_fall}
    else:
        class_weight = None
    
    # Compile
    # Chuẩn bị mô hình, định nghĩa cách mô hình học
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 
                 keras.metrics.Precision(name='precision'),
                 keras.metrics.Recall(name='recall')]
    )
    
    # Callbacks
    # Các hàm hỗ trợ quá trình giám sát, điều chỉnh học tập
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=8,
            min_lr=1e-7,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            f'best_{model_name.lower()}.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=0
        )
    ]
    
    # Train
    history = model.fit(
        train_ds,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_ds,
        validation_steps=validation_steps,
        epochs=EPOCHS,
        callbacks=callbacks,
        class_weight=class_weight,
        verbose=1
    )
    
    # Fine-tuning for transfer learning models
    if base_model is not None:
        print(f"\n🔧 Fine-tuning {model_name}...")
        base_model.trainable = True
        
        # Freeze early layers
        for layer in base_model.layers[:-30]:
            layer.trainable = False
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.0001),
            loss='binary_crossentropy',
            metrics=['accuracy', 
                     keras.metrics.Precision(name='precision'),
                     keras.metrics.Recall(name='recall')]
        )
        
        history_fine = model.fit(
            train_ds,
            steps_per_epoch=steps_per_epoch,
            validation_data=val_ds,
            validation_steps=validation_steps,
            epochs=30,
            callbacks=callbacks,
            class_weight=class_weight,
            verbose=1
        )
        
        # Merge histories
        for key in history.history.keys():
            history.history[key].extend(history_fine.history[key])
    
    # Plot
    plot_training_history(history, model_name)
    
    # Evaluate
    val_results = model.evaluate(val_ds, steps=validation_steps, verbose=0)
    
    return model, history, val_results


Tối ưu mô hình để deploy thông qua tflite

In [ ]:
def convert_to_tflite(model, model_name):
    """Chuyển đổi sang TFLite"""
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    # Lấy tập mẫu ngẫu nhiên để tính giá trị scale và zero-point để chuyển đổi sang INT8
    def representative_dataset():
        for _ in range(100):
            data = np.random.rand(1, IMG_SIZE, IMG_SIZE, 3).astype(np.float32)
            yield [data]
    
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    
    tflite_model = converter.convert()
    
    output_path = f'{model_name.lower()}_model.tflite'
    with open(output_path, 'wb') as f:
        f.write(tflite_model)
    
    size_kb = len(tflite_model) / 1024
    
    return output_path, size_kb

In [ ]:
if __name__ == "__main__":
    DATA_DIR = "D:\\UET courses\\IOT and Application\\Fall_detection\\Data\\images"
    
    if not os.path.exists(DATA_DIR):
        print(f" ERROR: Thư mục {DATA_DIR} không tồn tại!")
        print("\n Cấu trúc cần có:")
        print("images/")
        print("  ├── train/")
        print("  │   ├── fall*.jpg")
        print("  │   └── not_fall*.jpg")
        print("  └── valid/")
        print("      ├── fall*.jpg")
        print("      └── not_fall*.jpg")
    else:
        # Danh sách các mô hình để thử
        models_to_try = [
            'MobileNetV2',     # RECOMMENDED - tốt nhất cho ESP32
            'CustomCNN',       # Cân bằng
            'EfficientNetB0',  # Chính xác cao nhưng lớn hơn
            'SimpleCNN'        # Nhỏ nhất nhưng có thể kém chính xác
        ]
        
        results = []
        
        print("\n" + "="*60)
        print("🔬 EXPERIMENT: So sánh các kiến trúc mô hình")
        print("="*60)
        
        for model_name in models_to_try:
            try:
                # Train
                model, history, val_results = train_model(model_name, DATA_DIR)
                
                # Save
                model.save(f'{model_name.lower()}_model.h5')
                
                # Convert to TFLite
                tflite_path, size_kb = convert_to_tflite(model, model_name)
                
                # Store results
                val_acc = val_results[1]  # accuracy
                val_precision = val_results[2] if len(val_results) > 2 else 0
                val_recall = val_results[3] if len(val_results) > 3 else 0
                
                results.append({
                    'name': model_name,
                    'val_acc': val_acc,
                    'precision': val_precision,
                    'recall': val_recall,
                    'size_kb': size_kb,
                    'tflite_path': tflite_path
                })
                
                print(f"\n✅ {model_name} hoàn thành!")
                print(f"   Accuracy: {val_acc:.4f}")
                print(f"   Size: {size_kb:.2f} KB\n")
                
            except Exception as e:
                print(f"\n❌ Lỗi khi train {model_name}: {str(e)}\n")
                continue
        
        # So sánh kết quả
        print("\n" + "="*60)
        print("📊 KẾT QUẢ SO SÁNH")
        print("="*60)
        print(f"{'Model':<20} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'Size (KB)':<12}")
        print("-"*60)
        
        for r in sorted(results, key=lambda x: x['val_acc'], reverse=True):
            print(f"{r['name']:<20} {r['val_acc']:<12.4f} {r['precision']:<12.4f} {r['recall']:<12.4f} {r['size_kb']:<12.2f}")
        
        print("="*60)
        
        # Recommendation
        if results:
            best_acc = max(results, key=lambda x: x['val_acc'])
            best_size = min(results, key=lambda x: x['size_kb'])
            
            print("\n🏆 KHUYẾN NGHỊ:")
            print(f"  • Chính xác nhất: {best_acc['name']} (Acc: {best_acc['val_acc']:.4f})")
            print(f"  • Nhỏ nhất: {best_size['name']} (Size: {best_size['size_kb']:.2f} KB)")
            print(f"\n💡 Để triển khai ESP32, dùng: {best_acc['tflite_path']}")


🔬 EXPERIMENT: So sánh các kiến trúc mô hình

🚀 TRAINING: MobileNetV2


TRAINING SET: 748 images
  - Fall: 416 (55.6%)
  - Not Fall: 332 (44.4%)

VALIDATION SET: 222 images
  - Fall: 140 (63.1%)
  - Not Fall: 82 (36.9%)

2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


Model: "MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_0.35_128            │ (None, 4, 4, 1280)     │       410,208 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │        40,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 451,233 (1.72 MB)

 Trainable params: 41,025 (160.25 KB)

...

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpq9qbipll\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpq9qbipll\assets


Saved artifact at 'C:\Users\Admin\AppData\Local\Temp\tmpq9qbipll'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_494')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1516722714192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722715536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722715344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722714960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722716112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516231214480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722715728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722714576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722714384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1516722715152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(



✅ SimpleCNN hoàn thành!
   Accuracy: 0.7404
   Size: 30.01 KB


📊 KẾT QUẢ SO SÁNH
Model                Accuracy     Precision    Recall       Size (KB)   
------------------------------------------------------------
MobileNetV2          0.8413       0.9023       0.8571       647.43      
SimpleCNN            0.7404       0.7792       0.8571       30.01       
CustomCNN            0.7308       0.7442       0.9143       181.66      
EfficientNetB0       0.7163       0.8785       0.6714       4835.48     

🏆 KHUYẾN NGHỊ:
  • Chính xác nhất: MobileNetV2 (Acc: 0.8413)
  • Nhỏ nhất: SimpleCNN (Size: 30.01 KB)

💡 Để triển khai ESP32, dùng: mobilenetv2_model.tflite
